In [1]:
import syft
import syft.nn as nn
from syft.controller import tensors, models
import imp
imp.reload(syft.controller)
imp.reload(syft.nn)
imp.reload(syft)

import numpy as np
from syft import FloatTensor
import torch
from torch.autograd import Variable

In [2]:
input = FloatTensor([[0,0,1],[0,1.0,1],[1,0,1],[1,1,1]], autograd=True)
target = FloatTensor([[0],[0],[1],[1]]).autograd(True)
grad = FloatTensor([[1],[1],[1],[1]]).autograd(False)

np.random.seed(1)

weights1 = FloatTensor(np.random.rand(3,4)).autograd(True)
weights2 = FloatTensor(np.random.rand(4,1)).autograd(True)

input_t = Variable(torch.FloatTensor(input.to_numpy()), requires_grad=True)
target_t = Variable(torch.FloatTensor(target.to_numpy()), requires_grad=True)
weights1_t = Variable(torch.FloatTensor(weights1.to_numpy()), requires_grad=True)
weights2_t = Variable(torch.FloatTensor(weights2.to_numpy()), requires_grad=True)

grad_t = Variable(torch.FloatTensor(grad.to_numpy()))

In [3]:
weights1_t.data


 0.4170  0.7203  0.0001  0.3023
 0.1468  0.0923  0.1863  0.3456
 0.3968  0.5388  0.4192  0.6852
[torch.FloatTensor of size 3x4]

In [4]:
weights2_t.data


 0.2045
 0.8781
 0.0274
 0.6705
[torch.FloatTensor of size 4x1]

In [5]:
weights1_t.grad = None
weights2_t.grad = None
target_t.grad = None
input_t.grad = None

layer_1_t = input_t.mm(weights1_t).sigmoid()
layer_2_t = layer_1_t.mm(weights2_t).sigmoid()

diff_t = layer_2_t - target_t
loss_t = diff_t ** 2
loss_t.backward(grad_t)

#print(loss_t.sum().data[0])

weights1_t.data -= weights1_t.grad.data
weights2_t.data -= weights2_t.grad.data
weights1_t.grad.data


1.00000e-02 *
 -0.5600 -1.9540 -0.0849 -1.6177
  1.0343  4.5097  0.1307  2.8359
  2.1014  9.1505  0.2678  6.0765
[torch.FloatTensor of size 3x4]

In [6]:
weights2_t.grad.data


 0.2449
 0.2493
 0.2613
 0.2852
[torch.FloatTensor of size 4x1]

In [7]:
layer_1 = input.mm(weights1).sigmoid()
layer_2 = layer_1.mm(weights2).sigmoid()

diff = (layer_2 - target)
loss = diff ** 2 # Mean Squared Error Loss

loss.backward(grad)
weights1 -= weights1.grad()
weights2 -= weights2.grad()

In [8]:
weights1.grad()


[[-0.00559968 -0.01954043 -0.00084936 -0.01617729]
 [ 0.01034258  0.04509664  0.00130671  0.02835879]
 [ 0.02101371  0.09150549  0.00267767  0.06076522]]
[syft.FloatTensor:72 grad:None size:3x4 c:[] p:[70, 68] init:mm]

	-----------creators-----------
	[syft.FloatTensor:70 grad:None size:3x4 c:[72] p:[38] init:transpose]
	[syft.FloatTensor:68 grad:None size:4x4 c:[71, 72] p:[67, 62] init:mul_elem]
	------------------------------



In [9]:
weights2.grad()

[[ 0.244891 ]
 [ 0.249277 ]
 [ 0.2613002]
 [ 0.2851809]]
[syft.FloatTensor:73 grad:None size:4x1 c:[] p:[61, 59] init:mm]

	-----------creators-----------
	[syft.FloatTensor:61 grad:None size:4x4 c:[73] p:[44] init:transpose]
	[syft.FloatTensor:59 grad:None size:4x1 c:[62, 73] p:[58, 53] init:mul_elem]
	------------------------------

